# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,14.29,69,92,0.45,AU,1733692857
1,1,ancud,-41.8697,-73.8203,13.17,61,98,4.32,CL,1733692858
2,2,polyarnyye zori,67.3661,32.4981,-3.52,94,100,3.42,RU,1733692859
3,3,fomboni,-12.2800,43.7425,27.86,71,13,0.90,KM,1733692861
4,4,dalbandin,28.8947,64.4101,5.31,27,1,2.45,PK,1733692834


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.9,
    alpha = 0.7,
    color = "City"
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 24) & (city_data_df["Max Temp"] > 18) & (city_data_df['Cloudiness'] == 0), :]


# Drop any rows with null values
ideal_city_data_df.dropna(how="any")

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,christchurch,-43.5333,172.6333,23.11,21,0,5.66,NZ,1733692860
72,72,pacific grove,36.6177,-121.9166,18.82,71,0,4.02,US,1733692943
81,81,fabens,31.5023,-106.1586,18.55,23,0,10.18,US,1733692954
165,165,el granada,37.5027,-122.4694,20.07,63,0,6.17,US,1733693050
206,206,lompoc,34.6391,-120.4579,19.52,61,0,6.69,US,1733693097
256,256,benjamin hill,30.1667,-111.1000,22.82,17,0,3.06,MX,1733693157
281,281,long beach,33.7670,-118.1892,20.06,69,0,3.60,US,1733692900
311,311,crane,31.3974,-102.3501,21.35,19,0,11.58,US,1733693211
312,312,stephenville,32.2207,-98.2023,18.07,63,0,4.63,US,1733693224
317,317,chandler,33.3062,-111.8412,23.94,29,0,2.06,US,1733693230


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,christchurch,NZ,-43.5333,172.6333,21,
72,pacific grove,US,36.6177,-121.9166,71,
81,fabens,US,31.5023,-106.1586,23,
165,el granada,US,37.5027,-122.4694,63,
206,lompoc,US,34.6391,-120.4579,61,
256,benjamin hill,MX,30.1667,-111.1000,17,
281,long beach,US,33.7670,-118.1892,69,
311,crane,US,31.3974,-102.3501,19,
312,stephenville,US,32.2207,-98.2023,63,
317,chandler,US,33.3062,-111.8412,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "filter":"",
    "bias":"",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
christchurch - nearest hotel: No hotel found
pacific grove - nearest hotel: No hotel found
fabens - nearest hotel: No hotel found
el granada - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
benjamin hill - nearest hotel: No hotel found
long beach - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
chandler - nearest hotel: No hotel found
yanchep - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
puerto penasco - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
43,christchurch,NZ,-43.5333,172.6333,21,No hotel found
72,pacific grove,US,36.6177,-121.9166,71,No hotel found
81,fabens,US,31.5023,-106.1586,23,No hotel found
165,el granada,US,37.5027,-122.4694,63,No hotel found
206,lompoc,US,34.6391,-120.4579,61,No hotel found
256,benjamin hill,MX,30.1667,-111.1000,17,No hotel found
281,long beach,US,33.7670,-118.1892,69,No hotel found
311,crane,US,31.3974,-102.3501,19,No hotel found
312,stephenville,US,32.2207,-98.2023,63,No hotel found
317,chandler,US,33.3062,-111.8412,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.9,
    alpha = 0.7,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)